# 替换同型字

@author 何俊傑

In [1]:
from tqdm import tqdm
import random #随机抽取用

## 三、开发替换功能

### 3.字级别的形近替换

step1. 开发具体函数 get_shape_relavant，使得可以输入字后输出同形字列表

In [2]:
def read_from_charactorfile(f): ###对read_from_file做小小改动
    """从文件f中读取内容，并返回每一行对应的拼音和对应词，假设文件的编码是utf-8编码"""
    content = ''
    with open(f, 'r', encoding='utf-8') as f:
        content = f.readlines()    
    homophones = [line.split(' ') for line in content] #先把所有词读进来
    
    for line in homophones: #把换行符去掉的循环
        if line[-1][-1]=='\n':
            line[-1]=line[-1][:-1]
    ###主要改动在这行的‘list(set(line[1:][0]))’，保证了输出格式仍保持为key<字符>: value<字符集合>
    homophones = {line[0]: list(set(line[1:][0])) for line in homophones}        
    return homophones

In [3]:
###输入字，便可输出同形字集
def get_shape_relavant(charactor, shape_dict=read_from_charactorfile('./形近字语料库.txt')):
    """输入一个单字，可以输出其同形字列表。若词库中没有此词，便输出'no match word'"""
    msg = ''
    
    try:
        charactors = shape_dict[charactor]
        if len(charactors)==0:
            msg = 'no relavant charactor' #如果形近字集合数量为0，便输出'no relavant charactor'
            return msg
        elif charactor in charactors:
            charactors.remove(charactor)
            charactors = charactors[:10] #只输出前十个同形字，减少词库压力

    except KeyError:
        msg = 'no match charactor' #如果词库中没有此词语，便输出'no match word'
    
    if msg!='':
        return msg
    else:
        return charactors

In [4]:
shape_relevant = read_from_charactorfile('./形近字语料库.txt')

get_shape_relavant('吖'), get_shape_relavant('夏'), get_shape_relavant('天')

(['吩', '吟'], ['耍', '覃', '要', '雯'], 'no relavant charactor')

step2. 开发具体函数charactor_relavant_shape_replace，在输入句子后输出替换同形字后的结果

In [5]:
def charactor_relavant_shape_replace(sentence, relavant_shapes=read_from_charactorfile('./形近字语料库.txt')):
    sentence_tobechanged = list(sentence)
    for i in range(100):
        random_index = random.choice(range(len(sentence_tobechanged)))
        charactor = sentence[random_index]
        relavent_shape_charactors = get_shape_relavant(charactor, relavant_shapes) #获取同形字集合
        msg = type(relavent_shape_charactors)==list #判断函数返回的是不是同型字列表
        
        if msg:
            relavent_shape_charactor = random.choice(relavent_shape_charactors)
            sentence_tobechanged[random_index] = relavent_shape_charactor
            sentence_changed = "".join(sentence_tobechanged)
            return sentence_changed
        else:
            continue

In [6]:
sentence = '宋代定窑瓷器常见的器型以碗、盘、瓶'
charactor_relavant_shape_replace(sentence, shape_relevant)

'宋代定密瓷器常见的器型以碗、盘、瓶'

## 处理文本

In [7]:
def read_json_txt(path):
    result=[]
    import json
    with open(path, 'r',encoding='utf-8') as f:
        lines = f.readlines()
    for line in tqdm(lines):
        try:
            result.append(json.loads(line))
        except:
            continue
        
    return result

In [8]:
#字型错误 charactor_relavant_shape_replace(sentence)
path = './形近错别字.txt'
result = read_json_txt(path)

result_charactor_relavant_shape_replace = []

for sentence in tqdm(result):
    try:
        change = {'通畅':sentence['text'],'不通畅':charactor_relavant_shape_replace(sentence['text'])}
        result_charactor_relavant_shape_replace.append(change)
    except:
        continue

import json
with open('./形近错别字(处理完).txt', 'w', encoding='UTF-8') as f:
    for i in tqdm(result_charactor_relavant_shape_replace):
        f.write(json.dumps(i, ensure_ascii=False)+"\n")

100%|██████████| 199395/199395 [00:01<00:00, 107983.36it/s]
